In [1]:
from src.data import InputDataTrain
# from src.predict.py
import pandas as pd
import pickle
import src.utils as utils
import os
from catboost import Pool, cv, CatBoostRegressor


In [2]:
data_class = InputDataValidate("data")
with open("algorithm_data/algorithm_data.pkl", 'rb') as h:
    algorithm_data = pickle.load(h)

train_data = data_class.get_train()
train_friends = data_class.get_friends()
train_groups = data_class.get_groups()
graduations = algorithm_data['graduations']

In [3]:
group_dummies, top_groups, mlb = utils.calculate_group_dummies(train_data.drop(columns=['age', 'registered_year']), train_groups)

In [4]:
group_dummies

,uid,6,16,33,1761,1766,1768,1773,1775,1780,...,356718,356719,356724,356740,358502,358513,358517,360209,360212,360216
0,18976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,60390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,64114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29434,34181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29435,20963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29436,100904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29437,72117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
groups_data = train_groups.copy()
quantity_top_groups = 1000
top_groups = groups_data['gid'].value_counts().head(quantity_top_groups).index

groups_data_limited = groups_data[groups_data['gid'].isin(top_groups)]

In [ ]:
pd.get_dummies(groups_data_limited, columns=['gid']).groupby(['uid'], as_index=False).max()

In [ ]:
a = groups_data_limited.groupby('uid').agg({'gid': list}, axis=0)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer(sparse_output=True)

df = a.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(a['gid']),
                index=a.index,
                columns=mlb.classes_))

In [ ]:
mlb.transform([[32333334341234123123]])

In [ ]:
groups_data_limited[groups_data_limited['uid'] == 1]

In [ ]:
groups_data_limited.groupby('uid').to_dict()['gid']

In [ ]:
from sklearn.feature_extraction import FeatureHasher
enc = FeatureHasher()
a = enc.transform(groups_data_limited.set_index('uid'))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False, cols=['gid'])
a = enc.fit_transform(groups_data_limited )

In [ ]:
groups_data_limited

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)
OneHotEncoder(handle_unknown='ignore')
>>> enc.categories_
[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
>>> enc.transform([['Female', 1], ['Male', 4]]).toarray()
array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])
>>> enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])
array([['Male', 1],
       [None, 2]], dtype=object)
>>> enc.get_feature_names(['gender', 'group'])
array(['gender_Female', 'gender_Male', 'group_1', 'group_2', 'group_3'],
  dtype=object)

In [2]:
def train_model(data_class, algorithm_data_path):
    params = {"iterations": 2000,
        "depth": 6,
        "loss_function": "RMSE",
        "verbose": True,
        "early_stopping_rounds": 5}

    train = data_class.get_train()
    train_no_age = train.drop(columns=['age', 'registered_year'])
    education = data_class.get_education()
    friends = data_class.get_friends()
    groups_data = data_class.get_groups()
    graduations = utils.calcualte_graduations(train, education)

    train_groups_quant = utils.calculate_quantity(train_no_age, groups_data)
    train_groups_quant = train_groups_quant[['uid', 'gid']].rename(columns={'gid': 'qroups_quant'})

    train_friends_quant = utils.calculate_quantity(train_no_age, friends)
    train_friends_quant = train_friends_quant[['uid', 'fuid']].rename(columns={'fuid': 'friends_quant'})

    group_dummies = utils.calculate_group_dummies(train_no_age, groups_data)

    age_by_graduation = utils.calcualate_age_by_graduation(train_no_age, education, graduations)


    train_friens_age, mean_age_by_friends_dict = utils.calculate_friends_age(train_no_age, age_by_graduation, friends)



    mean_age_by_graduation = age_by_graduation['age_by_graduation'].mean()
    train_age_by_graduation = train_no_age.merge(age_by_graduation, on='uid', how='left')

    age_by_graduation['age_by_graduation'] = age_by_graduation['age_by_graduation'].fillna(mean_age_by_graduation)

    assert train.shape[0] ==\
        train_groups_quant.shape[0] ==\
        train_friends_quant.shape[0] ==\
        group_dummies.shape[0] ==\
        train_age_by_graduation.shape[0] ==\
        train_friens_age.shape[0]

    full_train = train.merge(train_groups_quant, on='uid').merge(train_friends_quant, on='uid').merge(group_dummies, on='uid').merge(train_age_by_graduation, on='uid').merge(train_friens_age, on='uid')


    cv_dataset = Pool(data=full_train.drop(columns=['age', 'uid']),
                    label=full_train['age'])
    scores = cv(cv_dataset,
                params,
                fold_count=4, 
                plot="True")

    params["iterations"] = scores['iterations'].max()

    cat = CatBoostRegressor(**params)

    full_train_to_model = full_train.drop(columns=['age', 'uid'])

    model = cat.fit(full_train_to_model, full_train['age'])

    algorithm_data = {}
    algorithm_data['graduations'] = graduations
    algorithm_data['mean_age_by_graduation'] = mean_age_by_graduation
    algorithm_data['top_groups'] = group_dummies.columns[1: ]
    algorithm_data['dataset_columns'] = full_train_to_model.columns
    algorithm_data['model'] = model
    algorithm_data['mean_age_by_friends_dict'] = mean_age_by_friends_dict

    with open(os.path.join(algorithm_data_path, "algorithm_data.pkl"), 'wb') as handle:
        pickle.dump(algorithm_data, handle)
    
    return full_train

full_train = train_model(data_class, 'algorithm_data')


NameError: name 'data_class' is not defined

In [ ]:
algorithm_data_path = 'algorithm_data/'
with open(os.path.join(algorithm_data_path, "algorithm_data.pkl"), 'rb') as handle:
    algorithm_data = pickle.load(handle)

data = pd.DataFrame({'feature_importance': algorithm_data['model'].get_feature_importance(), 
              'feature_names': full_train.drop(columns=['age', 'uid']).columns}).sort_values(by=['feature_importance'], 
                                                       ascending=False)

data.to_excel('tmp.xlsx')

In [ ]:
def predict(data_class, algorithm_data_path, result_save_path='/var/log/result'):
 
    test = data_class.get_test()
    test_no_reg_date = test.drop(columns='registered_year')
    education = data_class.get_education()
    friends = data_class.get_friends()
    groups_data = data_class.get_groups()

    with open(os.path.join(algorithm_data_path, "algorithm_data.pkl"), 'rb') as handle:
        algorithm_data = pickle.load(handle)
    
    graduations = algorithm_data['graduations']
    top_groups = algorithm_data['top_groups']
    mean_age_by_graduation = algorithm_data['mean_age_by_graduation']
    mean_age_by_friends_dict = algorithm_data['mean_age_by_friends_dict']
    model = algorithm_data['model']
    dataset_columns = algorithm_data['dataset_columns']

    test_groups_quant = utils.calculate_quantity(test_no_reg_date, groups_data)
    test_groups_quant = test_groups_quant[['uid', 'gid']].rename(columns={'gid': 'qroups_quant'})

    test_friends_quant = utils.calculate_quantity(test_no_reg_date, friends)
    test_friends_quant = test_friends_quant[['uid', 'fuid']].rename(columns={'fuid': 'friends_quant'})

    group_dummies = utils.calculate_group_dummies(test_no_reg_date, groups_data, top_groups)

    age_by_graduation = utils.calcualate_age_by_graduation(test_no_reg_date, education, graduations)



    age_by_graduation['age_by_graduation'] = age_by_graduation['age_by_graduation'].fillna(mean_age_by_graduation)


    test_friens_age, _ = utils.calculate_friends_age(test_no_reg_date, age_by_graduation, friends, mean_age_by_friends_dict)


    test_age_by_graduation = test_no_reg_date.merge(age_by_graduation, on='uid', how='left')

    test_age_by_graduation['age_by_graduation'] = test_age_by_graduation['age_by_graduation'].fillna(mean_age_by_graduation)

    assert test.shape[0] ==\
           test_groups_quant.shape[0] ==\
           test_friends_quant.shape[0] ==\
           group_dummies.shape[0] ==\
           test_age_by_graduation.shape[0] ==\
           test_friens_age.shape[0]

    full_test = test.merge(test_groups_quant, on='uid').merge(test_friends_quant, on='uid').merge(group_dummies, on='uid').merge(test_age_by_graduation, on='uid').merge(test_friens_age, on='uid')

    output = full_test[['uid']]
    output['age'] = model.predict(full_test[dataset_columns])

    
    return full_test, output

full_test, output = predict(data_class, 'algorithm_data', 'tmp')

output

In [ ]:
full_test

In [ ]:
true_target = data_class.get_true_test_target()

In [ ]:
import numpy as np
res = output.merge(true_target, on='uid')
np.sqrt(((res['age_x'] - res['age_y']) ** 2).mean())

In [3]:
full_test = pd.read_excel('full_test.xlsx')
output_ = pd.read_excel('output_.xlsx')
full_train = pd.read_pickle('full_train.pkl')

In [4]:
full_test

,Unnamed: 0,uid,registered_year,qroups_quant,friends_quant,6,16,33,1761,1766,...,age_by_friends1,age_by_friends2,age_by_friends3,age_by_friends4,age_by_friends5,age_by_friends6,age_by_friends7,age_by_friends8,age_by_friends9,age_by_friends10
0,0,85788,2020,4,20,0,0,0,0,0,...,34.849466,42.209914,38.053972,38.053972,38.053972,38.053972,38.053972,38.053972,38.053972,38.053972
1,1,19171,2009,100,92,0,0,0,0,0,...,29.172905,38.051235,36.759437,36.759437,36.759437,36.759437,36.759437,36.759437,36.759437,36.759437
2,2,7780,2012,0,26,0,0,0,0,0,...,34.849466,38.680198,37.233404,37.233404,37.233404,37.233404,37.233404,37.233404,37.233404,37.233404
3,3,14998,2019,0,1,0,0,0,0,0,...,34.849466,19.391405,19.391405,19.391405,19.391405,19.391405,19.391405,19.391405,19.391405,19.391405
4,4,94913,2010,0,19,0,0,0,0,0,...,34.849466,35.121662,35.121662,35.121662,35.121662,35.121662,35.121662,35.121662,35.121662,35.121662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,898,24062,2010,2,9,0,0,0,0,0,...,34.849466,36.366586,33.853779,33.935992,33.935992,33.935992,33.935992,33.935992,33.935992,33.935992
899,899,4763,2019,0,0,0,0,0,0,0,...,34.849466,34.563538,34.551572,34.551355,34.551355,34.551355,34.551355,34.551355,34.551355,34.551355
900,900,16188,2009,0,13,0,0,0,0,0,...,34.849466,38.809173,35.289546,35.289546,35.289546,35.289546,35.289546,35.289546,35.289546,35.289546
901,901,54565,2009,14,31,0,0,0,0,0,...,34.849466,39.677466,39.677466,39.677466,39.677466,39.677466,39.677466,39.677466,39.677466,39.677466


In [3]:
# import pickle
# import ./utils
# import os
# from data import InputDataTrain
# from catboost import Pool, cv, CatBoostRegressor



# def train_model(data_class, algorithm_data_path="algorithm_data/"):
#     train = data_class.get_train()
#     education = data_class.get_education()
#     friends = data_class.get_friends()
#     graduations = utils.calcualte_graduations(train, education)

#     algorithm_data = {}
#     algorithm_data['train_data'] = train
#     algorithm_data['train_friends'] = friends
#     algorithm_data['graduations'] = graduations

#     with open(os.path.join(algorithm_data_path, "algorithm_data.pkl"), 'wb') as handle:
#         pickle.dump(algorithm_data, handle)

#     return train


def train_model(data_class, algorithm_data_path):
    params = {"iterations": 2500,
        "depth": 1,
        "loss_function": "RMSE",
        "verbose": True,
        "early_stopping_rounds": 10}

    train = data_class.get_train()
    train_no_age = train.drop(columns=['age', 'registered_year'])
    education = data_class.get_education()
    friends = data_class.get_friends()
    groups_data = data_class.get_groups()
    graduations = utils.calcualte_graduations(train, education)

    train_groups_quant = utils.calculate_quantity(train_no_age, groups_data)
    train_groups_quant = train_groups_quant[['uid', 'gid']].rename(columns={'gid': 'qroups_quant'})

    train_friends_quant = utils.calculate_quantity(train_no_age, friends)
    train_friends_quant = train_friends_quant[['uid', 'fuid']].rename(columns={'fuid': 'friends_quant'})

    group_dummies, top_groups, mlb = utils.calculate_group_dummies(train_no_age, groups_data)

    age_by_graduation = utils.calcualate_age_by_graduation(train_no_age, education, graduations)


    train_friens_age, mean_age_by_friends_dict = utils.calculate_friends_age(train_no_age, age_by_graduation, friends)

    mean_age_by_graduation = age_by_graduation['age_by_graduation'].mean()
    train_age_by_graduation = train_no_age.merge(age_by_graduation, on='uid', how='left')

    age_by_graduation['age_by_graduation'] = age_by_graduation['age_by_graduation'].fillna(mean_age_by_graduation)

    assert train.shape[0] ==\
        train_groups_quant.shape[0] ==\
        train_friends_quant.shape[0] ==\
        group_dummies.shape[0] ==\
        train_age_by_graduation.shape[0] ==\
        train_friens_age.shape[0]

    full_train = train.merge(train_groups_quant, on='uid').merge(train_friends_quant, on='uid').merge(group_dummies, on='uid').merge(train_age_by_graduation, on='uid').merge(train_friens_age, on='uid')
    
    return full_train


data_class = InputDataTrain("./data")
full_train = train_model(data_class, "./algorithm_data/")

In [10]:

grid = {'learning_rate': [0.05, 0.1, 0.2],
        'depth': [8, 10, 12],
        'l2_leaf_reg': [3, 5, 4],
}

model = CatBoostRegressor(        iterations=2500,
        loss_function="RMSE",
        early_stopping_rounds=10)

randomized_search_result = model.randomized_search(grid,
                                                   X=full_train.drop(columns=['age', 'uid']),
                                                   y=full_train['age'],
                                                   n_iter=10,
                                                   plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 30.5751304	test: 30.6836727	best: 30.6836727 (0)	total: 147ms	remaining: 6m 7s
1:	learn: 25.2187218	test: 25.2920491	best: 25.2920491 (1)	total: 279ms	remaining: 5m 48s
2:	learn: 21.0406545	test: 21.0748827	best: 21.0748827 (2)	total: 408ms	remaining: 5m 39s
3:	learn: 17.8299918	test: 17.8364140	best: 17.8364140 (3)	total: 592ms	remaining: 6m 9s
4:	learn: 15.4106472	test: 15.4033293	best: 15.4033293 (4)	total: 720ms	remaining: 5m 59s
5:	learn: 13.6621184	test: 13.6509118	best: 13.6509118 (5)	total: 845ms	remaining: 5m 51s
6:	learn: 12.3594589	test: 12.3439010	best: 12.3439010 (6)	total: 950ms	remaining: 5m 38s
7:	learn: 11.3650703	test: 11.3400997	best: 11.3400997 (7)	total: 1.03s	remaining: 5m 20s
8:	learn: 10.7116363	test: 10.6900095	best: 10.6900095 (8)	total: 1.1s	remaining: 5m 5s
9:	learn: 10.2000129	test: 10.1761964	best: 10.1761964 (9)	total: 1.18s	remaining: 4m 53s
10:	learn: 9.8556224	test: 9.8325984	best: 9.8325984 (10)	total: 1.26s	remaining: 4m 45s
11:	learn: 9.62

In [ ]:
randomized_search_result['params']

{'depth': 8, 'l2_leaf_reg': 5, 'learning_rate': 0.1}